In [208]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import datetime as DT
import pickle
print('system: ', sys.version)
print('numpy: ', np.__version__)
print('panda: ', pd.__version__)

system:  3.5.1 |Anaconda 2.4.1 (64-bit)| (default, Dec  7 2015, 15:00:12) [MSC v.1900 64 bit (AMD64)]
numpy:  1.10.1
panda:  0.17.1


In [209]:
# Create Trend Ranges, based on visual inspection of previous graph, and add Trends to Order dataframe
trend_range = [0, 6, 33, 60, 77, 86, 150, 171, 207, 222, 271, 314, 331, 348]
trend_labels = ['UP1', 'FLAT1', 'DOWN1', 'UP2', 'DOWN2', 'FLAT2', 'DOWN3', 'UP3', 'DOWN4', 'UP4', 
               'DOWN5', 'UP6', 'FLAT3']

#Categorize order dataframe based on trend ranges
df_ord['Trend'] = pd.cut(df_ord.Ticket, trend_range, labels=trend_labels).astype('category')

In [210]:
with open('df_Ind.pickle', 'rb') as file:
    df_Ind = pickle.load(file)
    
with open('df_ord.pickle', 'rb') as file:
    df_ord = pickle.load(file)

In [211]:
# Create dataframe study to contain data that will be evaluated
study = df_Ind.loc[:, ('DateTime', 'Trend', 'BBAND(100)', 'MACD(12) MAIN', 'MACD(2) SIGNAL', 'MACD(10) SIGNAL', 'MACD(12) SIGNAL' , 'MACD(20) SIGNAL')]

# Add stats to study dataframe
rolling_window = 7
study['bband100_std'] = pd.rolling_std(study['BBAND(100)'], rolling_window)
study['macd12main_std'] = pd.rolling_std(study['MACD(12) MAIN'], rolling_window)
study['macd2signal_std'] = pd.rolling_std(study['MACD(2) SIGNAL'], rolling_window)
study['mcad2signal_std_ave'] = pd.rolling_mean(study['macd2signal_std'], rolling_window)
study['macd10signal_std'] = pd.rolling_std(study['MACD(10) SIGNAL'], rolling_window)
study['macd12signal_std'] = pd.rolling_std(study['MACD(12) SIGNAL'], rolling_window)
study['mcad12signal_std_ave'] = pd.rolling_mean(study['macd12signal_std'], rolling_window*3)
study['macd20signal_std'] = pd.rolling_std(study['MACD(20) SIGNAL'], rolling_window)

In [212]:
#define Gain and Loss filters
Gain = (df_ord['Profit'] > 0)
Loss = (df_ord['Profit'] < 0)

#define filters for trends
up1 = (study['Trend'] == 'UP1')
flat1 = (study['Trend'] == 'FLAT1')
down1 = (study['Trend'] == 'DOWN1')
up2 = (study['Trend'] == 'UP2')
down2 = (study['Trend'] == 'DOWN2')
flat2 = (study['Trend'] == 'FLAT2')
down3 = (study['Trend'] == 'DOWN3')
up3 = (study['Trend'] == 'UP3')
down4 = (study['Trend'] == 'DOWN4')
up4 = (study['Trend'] == 'UP4')
down5 = (study['Trend'] == 'DOWN5')
up6 = (study['Trend'] == 'UP6')
flat3 = (study['Trend'] == 'FLAT3')

In [213]:
# Pull Gain orders from Order dataframe
gain = df_ord.loc[Gain, ('DateTime', 'Trend', 'Ticket', 'Profit', 'Type')]
gain['PlotLevel'] = 0.00020
loss = df_ord.loc[Loss, ('DateTime', 'Trend', 'Ticket', 'Profit', 'Type')]
loss['PlotLevel'] = 0.00020

In [237]:
def plotmean(subdata, suby, subplt):
#    print(subdata[suby].head())
#    print(subdata['DateTime'].head())
    L = len(subdata)
#    print(L)
    A = pd.DataFrame({'DateTime':[subdata['DateTime'].iloc[0],
                                  subdata['DateTime'].iloc[L-1]],
                      'Average':[subdata[suby].mean(),
                                 subdata[suby].mean()]})
#    print(A)    
#    print(type(A))
    plt.subplot(subplt)
    plt.plot(list(A['DateTime']), list(A['Average']), color='grey', linestyle='-', marker=None)

    
def plotter(plotdata, **kwarg):  
    # iterate through argument list and assign appropriate chart data
    for kw, val in kwarg.items():
        color = 'blue'
        wt = 1
        zorder = 1
        linestyle = '-'
        marker = None
        if kw == 'sub':
            subplt = val
            plt.subplot(val)
        if kw == 'ylabel':
            plt.ylabel(val)
        if kw == 'xlabel':
            plt.xlabel(val)
        if kw == 'ymax':
            plt.ylim(ymax=val)
        if kw == 'title':
            plt.title(val) 
    #iterate through plot data and get information for each series to be plotted
    for series, itemlist in plotdata.items():
        color = 'blue'
        wt = 1
        zorder = 1
        linestyle = '-'
        marker = None
        plot_mean = False
        for item, val in itemlist.items():
            if item == 'ydata':
                ydata = val
            if item == 'dataframe':
                data = val   
            if item == 'color':
                color = val
            if item == 'wt':
                wt = val
            if item == 'zorder':
                zorder = val
            if item == 'linestyle':
                linestyle = val
            if item == 'marker':
                marker = val
            if item == 'plot_mean':
                plot_mean = True
        
        if plot_mean == True:
            plotmean(data, ydata, subplt)
        plt.plot(list(data['DateTime']),list(data[ydata]), color=color, zorder=zorder, marker=marker, linestyle=linestyle)

In [238]:
def plot(datarange, series, trend, subplot, color, title):
    plotter({series:{
             'ydata':series,
             'dataframe':datarange,
             'color':color,
             'plot_mean':True},
         
             'Series:Loss':{
             'ydata':'PlotLevel',
             'dataframe':loss.loc[(loss['Trend'] == trend)],
             'color':'r', 'marker':'o', 'linestyle':''}, 
         
             'Series:Gain':{
             'ydata':'PlotLevel',
             'dataframe':gain.loc[(gain['Trend'] == trend)],
             'color':'g', 'marker':'o', 'linestyle':''}}, 

             ylabel=series, sub=subplot, ymax=maxy, title=title)

In [242]:
maxy = 0.0004
nrows = 2
ncols = 1
sub = (nrows * 100) + (ncols * 10)

plt.figure(1)
# plotter sub 1
plot(study[(study['Trend'] == 'UP1')], 'macd12main_std', 'UP1', sub+1, 'b', 'Period UP1')


# plotter sub 2
plot(study[(study['Trend'] == 'UP1')], 'macd12signal_std', 'UP1', sub+2, 'r','Period UP1')

plt.figure(2)
# plotter sub 3
plot(study[(study['Trend'] == 'UP2')], 'macd12main_std', 'UP2', sub+1, 'b', 'Period UP2')
# plotter sub 4
plot(study[(study['Trend'] == 'UP2')], 'macd12signal_std', 'UP2', sub+2, 'r', 'Period UP2')
# # plotter sub 5
# plot(study[(study['Trend'] == 'UP3')], 'macd12main_std', 'UP3', sub+5, 'b')
# # plotter sub 6
# plot(study[(study['Trend'] == 'UP3')], 'macd12signal_std', 'UP3', sub+6, 'r')
# # plotter sub 7
# plot(study[(study['Trend'] == 'UP4')], 'macd12main_std', 'UP4', sub+7, 'b')
# # plotter sub 8
# plot(study[(study['Trend'] == 'UP4')], 'macd12signal_std', 'UP4', sub+8, 'r')

plt.figure(3)
# plotter sub 1
plot(study[(study['Trend'] == 'DOWN1')], 'macd12main_std', 'DOWN1', sub+1, 'b', 'Period DOWN1')
# plotter sub 2
plot(study[(study['Trend'] == 'DOWN1')], 'macd12signal_std', 'DOWN1', sub+2, 'r', 'Period DOWN1')
# # plotter sub 3
# plot(study[(study['Trend'] == 'DOWN2')], 'macd12main_std', 'DOWN2', sub+3, 'b')
# # plotter sub 4
# plot(study[(study['Trend'] == 'DOWN2')], 'macd12signal_std', 'DOWN2', sub+4, 'r')
# # plotter sub 5
# plot(study[(study['Trend'] == 'DOWN3')], 'macd12main_std', 'DOWN3', sub+5, 'b')
# # plotter sub 6
# plot(study[(study['Trend'] == 'DOWN3')], 'macd12signal_std', 'DOWN3', sub+6, 'r')
# # plotter sub 7
# plot(study[(study['Trend'] == 'DOWN4')], 'macd12main_std', 'DOWN4', sub+7, 'b')
# # plotter sub 8
# plot(study[(study['Trend'] == 'DOWN4')], 'macd12signal_std', 'DOWN4', sub+8, 'r')

plt.show()